In [1]:
import pandas as pd
import numpy as np

# Importando os dados já filtrados

In [2]:
cdr = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/cdr_after_filtes.csv')
cdr = cdr.drop('Unnamed: 0',1)
len(cdr)

8399068

# Importando a localização das antenas

In [3]:
#É possível um mesmo conjunto de LAT e LONG terem mais de um Id atrelado
antennas = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/antenas_regiao_imediata_sjdr.csv', error_bad_lines = False, sep=';')
antennas = antennas.rename(columns={'id': 'ID', 'antenna': 'ANTENNA', 'city': 'CITY', 'lat': 'LAT', 'long':'LONG'})
antennas.head()

,ID,ANTENNA,CITY,LAT,LONG
0,4593,27712,Madre de Deus de Minas,-21.48194,-44.33039
1,4706,23442,São João del-Rei,-21.10707,-44.21979
2,4707,23463,São João del-Rei,-21.12699,-44.26144
3,4863,23462,São João del-Rei,-21.12699,-44.26144
4,5001,29943,Tiradentes,-21.11886,-44.16478


In [4]:
#O arquivo unique_locations contém as localizações únicas das antenas
#Atribui um identiifcador único pra elas, afim de adicionar esse identificador no dataframe de antenas
antennas_unique = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/unique_locations.txt', sep=']', header=None)
antennas_unique[0] = antennas_unique[0].str.replace('[','')
ids = ['MDM', 'SJDR1', 'SJDR2', 'SJDR3', 'SJDR4', 'SJDR5','TR1', 'TR2', 'LD1', 'RT', 'SVM', 'ST']
antennas_unique['ANTENNA_ID'] = ids
antennas_unique = antennas_unique.rename(columns={0:'ANTENNA', 1: 'CITY'})
antennas_unique.head(13)

,ANTENNA,CITY,ANTENNA_ID
0,"27711,27712,27713,48582,27451,48581",Madre de Deus de Minas,MDM
1,"23441,23442,50153,23443,50152,50151",São João del-Rei,SJDR1
2,"23461,23463,23462,50933,50931,50932",São João del-Rei,SJDR2
3,"23471,23472,23473,51311,51312",São João del-Rei,SJDR3
4,"23451,23452,23453,50663,50661,50662",São João del-Rei,SJDR4
5,"50141,50142,50143",São João del-Rei,SJDR5
6,"29941,29943,53562,50911,49131,50913,29942,5356...",Tiradentes,TR1
7,"29011,29013,29012",Tiradentes,TR2
8,"49592,51041,51042,49593,51043",Lagoa Dourada,LD1
9,"51601,51603,51602",Ritápolis,RT


In [5]:
antennas_with_id = antennas.merge(antennas_unique.assign(ANTENNA=antennas_unique['ANTENNA'].str.split(','))
               .explode('ANTENNA').assign(ANTENNA=lambda x: x['ANTENNA'].astype(int)), on='ANTENNA')

antennas_with_id = antennas_with_id.drop('CITY_y', 1)
antennas_with_id = antennas_with_id.rename(columns={'CITY_x': 'CITY'})
antennas_with_id.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/antenas/antennas_with_id.csv', index=False)
antennas_with_id.head()

,ID,ANTENNA,CITY,LAT,LONG,ANTENNA_ID
0,4593,27712,Madre de Deus de Minas,-21.48194,-44.33039,MDM
1,4706,23442,São João del-Rei,-21.10707,-44.21979,SJDR1
2,4707,23463,São João del-Rei,-21.12699,-44.26144,SJDR2
3,4863,23462,São João del-Rei,-21.12699,-44.26144,SJDR2
4,5001,29943,Tiradentes,-21.11886,-44.16478,TR1


In [6]:
len(antennas_with_id)

57

In [7]:
cdr_merge_antennas = pd.merge(cdr, antennas_with_id[['ANTENNA', 'ANTENNA_ID']], on='ANTENNA', how='left')
cdr_merge_antennas.head(7)

,DATE,HOUR,DURATION,DDD_USER_FROM,USER_FROM,DDD_USER_TO,USER_TO,ANTENNA,8,9,10,11,12,13,14,DAY_OF_WEEK,ANTENNA_ID
0,2013-03-21,11:51:30,0.37,32,5B5F2C071D12AF13219DF5EBE05132AF,32,9FB3B96B6D5E16C9DD564AA3E84F1954,50141,NaN,117,LOCAL,OI,OI,PRE PAGO,300000533,Thursday,SJDR5
1,2013-03-21,21:03:30,0.13,61,B3299B0E587D7275E3E4D530E9EECF50,98,6432F1DF21BA38368D9A165C739EEBB3,51601,NaN,195,LONGA DISTANCIA,BRT,OI,PRE PAGO,300000210,Thursday,RT
2,2013-03-21,17:33:20,0.33,61,85D5C50A6D882CA8E4BB00BCA3574417,61,0D8583F810B9720A8032BB939F12B3FF,51601,NaN,117,LOCAL,BRT,BRT,PRE PAGO,300000210,Thursday,RT
3,2013-03-21,11:58:26,1.27,61,6C10A9E9F325CAA3CCB7F9A0D6983D2A,61,B0C50ED1DEC9E06E4C64E7419DDC4B09,51601,NaN,117,LOCAL,BRT,BRT,PRE PAGO,300000210,Thursday,RT
4,2013-03-21,16:06:59,0.17,61,29122DCF4189448865B03F24C2DFEAE6,61,E0B097D86BF9C02A23011610EBAF47FD,51601,NaN,77,LOCAL,BRT,GVT,PRE PAGO,300000210,Thursday,RT
5,2013-03-21,11:14:19,2.12,61,07B468660874AD7C8FE958741A768181,61,728614A7ED24715CC8502F6B4DD53491,51601,NaN,117,LOCAL,BRT,BRT,PRE PAGO,300000210,Thursday,RT
6,2013-03-21,20:30:29,7.02,61,BA4DEA1F710C1216F74F1255150C49D6,61,4F6DCBA36292B8487743E13319E67DF6,51601,NaN,77,LOCAL,BRT,GVT,PRE PAGO,300000210,Thursday,RT


In [8]:
presumed_residence = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/Regiao_Presumida/residence_antenna.csv', sep=';')
presumed_residence = presumed_residence.rename(columns={'user': 'USER','residence_antenna': 'ANTENNA'})
presumed_residence_with_id = pd.merge(presumed_residence, antennas_with_id[['ANTENNA', 'ANTENNA_ID']], 
                                      on='ANTENNA', how='left')

presumed_residence_with_id = presumed_residence_with_id.rename(columns={'USER': 'USER_FROM', 
                                                                        'ANTENNA_ID': 'PRESUMED_RESIDENCE'})
presumed_residence_with_id

,USER_FROM,ANTENNA,PRESUMED_RESIDENCE
0,00003F26576D1122848B6F3CEEC250E5,50141,SJDR5
1,000579E8BA1DA9E5292392473BCCCD2A,49592,LD1
2,0008962B966EF6C5571DF03F5C786677,23461,SJDR2
3,0008B7FB336065A53EC31AD06D712E28,51601,RT
4,0008B8E32E18BF4450B7CAAF0C1CE179,23441,SJDR1
...,...,...,...
59785,FFFA3339BA362F788F81DF22F3F7B6BA,27711,MDM
59786,FFFDA183352D8DD3AC2913B09BD4174F,23451,SJDR4
59787,FFFDBC5EE75F3127C262488987D6F99D,23441,SJDR1
59788,FFFE9C91DE90C5CA962FE90319E8D620,23451,SJDR4


In [9]:
merged = pd.merge(cdr_merge_antennas, presumed_residence_with_id[['USER_FROM', 'PRESUMED_RESIDENCE']], on='USER_FROM', how='right')
#df_to_distance = merged[['USER_FROM', 'ANTENNA_ID', 'PRESUMED_RESIDENCE']]
#df_to_distance = df_to_distance.rename(columns={'USER_FROM': 'USER'})
#df_to_distance.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/df_main.csv')
merged

,DATE,HOUR,DURATION,DDD_USER_FROM,USER_FROM,DDD_USER_TO,USER_TO,ANTENNA,8,9,10,11,12,13,14,DAY_OF_WEEK,ANTENNA_ID,PRESUMED_RESIDENCE
0,2013-04-03,17:09:03,0.80,83.0,00003F26576D1122848B6F3CEEC250E5,83.0,FFA5E22189ED7ADEB364CFEB18205B59,50141.0,NaN,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Wednesday,SJDR5,SJDR5
1,2013-04-09,15:34:30,5.12,83.0,00003F26576D1122848B6F3CEEC250E5,83.0,CCA21FFF10C7EC088827F5D5770D3274,50141.0,NaN,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Tuesday,SJDR5,SJDR5
2,2013-04-09,15:23:38,1.53,83.0,00003F26576D1122848B6F3CEEC250E5,-4.0,64142FC9CDA8A82085271FB9D25FCDBF,50141.0,NaN,299.0,SERVICO NAO GEOGRAFICO,OI,TNL,PRE PAGO,300000210.0,Tuesday,SJDR5,SJDR5
3,2013-04-10,16:26:16,0.35,83.0,00003F26576D1122848B6F3CEEC250E5,83.0,BD288CE0AB2A0B5E57EB635FBF814C80,50141.0,NaN,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Wednesday,SJDR5,SJDR5
4,2013-04-12,22:47:37,0.07,83.0,00003F26576D1122848B6F3CEEC250E5,83.0,8BA3ED7CA5B5E5D556AF30AE93D578FD,50141.0,NaN,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Friday,SJDR5,SJDR5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494937,2013-04-08,19:07:47,0.63,61.0,FFFF6761884073022FB5DFE1B6F54DE3,61.0,4685E8968E615A30082B35B68AAAE498,51601.0,NaN,117.0,LOCAL,BRT,BRT,PRE PAGO,300000210.0,Monday,RT,RT
1494938,2013-04-10,21:48:42,1.32,61.0,FFFF6761884073022FB5DFE1B6F54DE3,61.0,4685E8968E615A30082B35B68AAAE498,51601.0,NaN,117.0,LOCAL,BRT,BRT,PRE PAGO,300000210.0,Wednesday,RT,RT
1494939,2013-04-11,08:30:53,3.93,61.0,FFFF6761884073022FB5DFE1B6F54DE3,61.0,4685E8968E615A30082B35B68AAAE498,51601.0,NaN,117.0,LOCAL,BRT,BRT,PRE PAGO,300000210.0,Thursday,RT,RT
1494940,2013-04-13,18:40:09,2.57,61.0,FFFF6761884073022FB5DFE1B6F54DE3,61.0,7700C6F0C0CB1DA5518304D1E9FA31F5,51601.0,NaN,117.0,LOCAL,BRT,BRT,PRE PAGO,300000210.0,Saturday,RT,RT


In [10]:
merged.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/df_main_full.csv')

In [16]:
def generate_df(df1):
    merged = pd.merge(df1, presumed_residence_with_id[['USER_FROM', 'PRESUMED_RESIDENCE']], on='USER_FROM', how='right')
    df_to_distance = merged[['USER_FROM', 'ANTENNA_ID', 'PRESUMED_RESIDENCE']]
    df_to_distance = df_to_distance.rename(columns={'USER_FROM': 'USER'})
    df_to_distance.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/df_monday.csv')
    
    return df_to_distance

In [17]:
cdr_monday = cdr_merge_antennas.loc[cdr_merge_antennas['DAY_OF_WEEK'] == 'Monday']
x = generate_df(cdr_monday)
x

,USER,ANTENNA_ID,PRESUMED_RESIDENCE
0,00003F26576D1122848B6F3CEEC250E5,NaN,SJDR5
1,000579E8BA1DA9E5292392473BCCCD2A,LD1,LD1
2,000579E8BA1DA9E5292392473BCCCD2A,LD1,LD1
3,0008962B966EF6C5571DF03F5C786677,SJDR2,SJDR2
4,0008B7FB336065A53EC31AD06D712E28,NaN,RT
...,...,...,...
87004,FFFE9C91DE90C5CA962FE90319E8D620,SJDR4,SJDR4
87005,FFFE9C91DE90C5CA962FE90319E8D620,SJDR4,SJDR4
87006,FFFF6761884073022FB5DFE1B6F54DE3,RT,RT
87007,FFFF6761884073022FB5DFE1B6F54DE3,RT,RT
